In [2]:
%matplotlib inline

# python libraties
import os, cv2,itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
from PIL import Image

# pytorch libraries
import torch
from torch import optim,nn
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
from torchvision import models,transforms

# sklearn libraries
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# ensure results are reproducible
np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed(10)

In [3]:
data_dir = "/kaggle/input/aiml-general-championship"
print(os.listdir(data_dir))

['KCDH2024_Training_LesionGroupings.csv', 'KCDH2024_Test_Input', 'sample_submission.csv', 'KCDH2024_Training_Input_10K', 'KCDH2024_Training_GroundTruth.csv']


<h1>Loading Data... </h1>

In [4]:
img_data_dir = data_dir + r"/KCDH2024_Training_Input_10K/KCDH2024_Training_Input_10K"
all_image_path = glob(os.path.join(img_data_dir, '*.jpg'))
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}  # key - image filename,   value - path to image
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'dermatofibroma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [5]:
len(imageid_path_dict)

9600

In [6]:
img = cv2.imread(imageid_path_dict["ISIC_0024308"])
img.shape

(450, 600, 3)

In [7]:
def compute_img_mean_std(image_paths):
    """
        computing the mean and std of three channel on the whole dataset,
        first we should normalize the image from 0-255 to 0-1
    """

    img_h, img_w = 224, 224              # Size to resize..
    imgs = []
    means, stdevs = [], []

    for i in tqdm(range(len(image_paths))):
        img = cv2.imread(image_paths[i])
        img = cv2.resize(img, (img_h, img_w))
        imgs.append(img)

    imgs = np.stack(imgs, axis=3)
    print(imgs.shape)

    imgs = imgs.astype(np.float32) / 255.

    for i in range(3):
        pixels = imgs[:, :, i, :].ravel()  # resize to one row
        means.append(np.mean(pixels))
        stdevs.append(np.std(pixels))

    means.reverse()  # BGR --> RGB
    stdevs.reverse()

    print("normMean = {}".format(means))
    print("normStd = {}".format(stdevs))
    return means,stdevs

In [8]:
# means , stdevs = compute_img_mean_std(all_image_path)

In [9]:
# Store Values to save time in future..
norm_mean = [0.76696384, 0.54525656, 0.56884694]
norm_std = [0.13945772, 0.15192385, 0.16916788]

In [10]:
lesion_db = pd.read_csv(os.path.join(data_dir, 'KCDH2024_Training_LesionGroupings.csv'))
truth_db = pd.read_csv(os.path.join(data_dir, 'KCDH2024_Training_GroundTruth.csv'))

df = pd.merge(lesion_db, truth_db, on = 'image', how = 'inner')
df.drop("diagnosis_confirm_type", axis = 1, inplace = True)
df['path'] = df['image'].map(imageid_path_dict.get)
df

,image,lesion_id,MEL,NV,BCC,AKIEC,BKL,DF,VASC,path
0,ISIC_0024306,HAM_0000550,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
1,ISIC_0024307,HAM_0003577,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
2,ISIC_0024308,HAM_0001477,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
3,ISIC_0024309,HAM_0000484,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
4,ISIC_0024310,HAM_0003350,1,0,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
...,...,...,...,...,...,...,...,...,...,...
10010,ISIC_0034316,HAM_0004304,1,0,0,0,0,0,0,None
10011,ISIC_0034317,HAM_0006376,1,0,0,0,0,0,0,None
10012,ISIC_0034318,HAM_0000344,0,0,0,0,1,0,0,None
10013,ISIC_0034319,HAM_0000747,0,1,0,0,0,0,0,None


In [11]:
# Remove the rows not containg path to images..
df = df[df['path'].notna()]
df

,image,lesion_id,MEL,NV,BCC,AKIEC,BKL,DF,VASC,path
0,ISIC_0024306,HAM_0000550,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
1,ISIC_0024307,HAM_0003577,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
2,ISIC_0024308,HAM_0001477,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
3,ISIC_0024309,HAM_0000484,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
4,ISIC_0024310,HAM_0003350,1,0,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
...,...,...,...,...,...,...,...,...,...,...
9595,ISIC_0033901,HAM_0002342,1,0,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
9596,ISIC_0033902,HAM_0000048,1,0,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
9597,ISIC_0033903,HAM_0003367,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
9598,ISIC_0033904,HAM_0005820,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...


In [12]:
# Convert 7 different columns of different lesisons to single ...

cell_type_idx = []

for index, row in df.iterrows():
    cell_type_idx_row = row["MEL"], row["NV"], row["BCC"], row["AKIEC"], row["BKL"], row["DF"], row["VASC"]
    cell_type_idx.append(cell_type_idx_row.index(1))

# Assign a new column..
df["cell_type_idx"] = cell_type_idx

# Drop older columns..
df.drop( columns = ["MEL", "NV", "BCC", "AKIEC", "BKL", "DF", "VASC"], inplace = True)

df.head()

/tmp/ipykernel_34/2766090685.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cell_type_idx"] = cell_type_idx
/tmp/ipykernel_34/2766090685.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop( columns = ["MEL", "NV", "BCC", "AKIEC", "BKL", "DF", "VASC"], inplace = True)


,image,lesion_id,path,cell_type_idx
0,ISIC_0024306,HAM_0000550,/kaggle/input/aiml-general-championship/KCDH20...,1
1,ISIC_0024307,HAM_0003577,/kaggle/input/aiml-general-championship/KCDH20...,1
2,ISIC_0024308,HAM_0001477,/kaggle/input/aiml-general-championship/KCDH20...,1
3,ISIC_0024309,HAM_0000484,/kaggle/input/aiml-general-championship/KCDH20...,1
4,ISIC_0024310,HAM_0003350,/kaggle/input/aiml-general-championship/KCDH20...,0


In [13]:
# Determine how many images are associated with each lesion_id ?
df_undup = df.groupby('lesion_id').count()

# Filter out lesion_id's that have only one image associated with it
df_undup = df_undup[df_undup['image'] == 1]
df_undup.reset_index(inplace=True)
df_undup.head()

,lesion_id,image,path,cell_type_idx
0,HAM_0000001,1,1,1
1,HAM_0000003,1,1,1
2,HAM_0000004,1,1,1
3,HAM_0000007,1,1,1
4,HAM_0000008,1,1,1


In [14]:
df_undup = df.groupby('lesion_id').count()
df_undup.head()

# Filter out lesion_id's that have only one image associated with it
df_undup = df_undup[df_undup['image'] == 1]
df_undup.reset_index(inplace=True)
df_undup[:]

,lesion_id,image,path,cell_type_idx
0,HAM_0000001,1,1,1
1,HAM_0000003,1,1,1
2,HAM_0000004,1,1,1
3,HAM_0000007,1,1,1
4,HAM_0000008,1,1,1
...,...,...,...,...
5592,HAM_0007622,1,1,1
5593,HAM_0007623,1,1,1
5594,HAM_0007624,1,1,1
5595,HAM_0007626,1,1,1


In [15]:
# Identify lesion_id's that have duplicate images and those that have only one image.
def get_duplicates(x):
    unique_list = list(df_undup['lesion_id'])
    if x in unique_list:
        return 'unduplicated'
    else:
        return 'duplicated'

# Create a new colum that is a copy of the lesion_id column
df['duplicates'] = df['lesion_id']
# Apply the function to this new column
df['duplicates'] = df['duplicates'].apply(get_duplicates)
df.head()

/tmp/ipykernel_34/1363690245.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['duplicates'] = df['lesion_id']
/tmp/ipykernel_34/1363690245.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['duplicates'] = df['duplicates'].apply(get_duplicates)


,image,lesion_id,path,cell_type_idx,duplicates
0,ISIC_0024306,HAM_0000550,/kaggle/input/aiml-general-championship/KCDH20...,1,unduplicated
1,ISIC_0024307,HAM_0003577,/kaggle/input/aiml-general-championship/KCDH20...,1,unduplicated
2,ISIC_0024308,HAM_0001477,/kaggle/input/aiml-general-championship/KCDH20...,1,unduplicated
3,ISIC_0024309,HAM_0000484,/kaggle/input/aiml-general-championship/KCDH20...,1,unduplicated
4,ISIC_0024310,HAM_0003350,/kaggle/input/aiml-general-championship/KCDH20...,0,duplicated


In [16]:
df['duplicates'].value_counts()

duplicates
unduplicated    5597
duplicated      4003
Name: count, dtype: int64

In [17]:
# Filter out images that don't have duplicates (count = 1)  (Removed augmented images)
# We will use this data to get validation set..
df_undup = df[df['duplicates'] == 'unduplicated']
df_undup.shape

(5597, 5)

In [18]:
# Create a val set using df as none of these images have augmented duplicates in the training set now..
y = df_undup['cell_type_idx']
_, df_val = train_test_split(df_undup, test_size=0.2, random_state=101, stratify=y)
df_val.shape

(1120, 5)

In [19]:
df_val['cell_type_idx'].value_counts()

cell_type_idx
1    891
4     89
0     52
2     37
3     30
6     13
5      8
Name: count, dtype: int64

In [20]:
# Remove the validation rows from the original data to get training rows..

# This function identifies if an image is part of the train or val set.
def get_val_rows(x):
    # create a list of all the lesion_id's in the val set
    val_list = list(df_val['image'])
    if str(x) in val_list:
        return 'val'
    else:
        return 'train'

# Identify train and val rows..
# Create a new colum that is a copy of the image column
df['train_or_val'] = df['image']

# Apply the function to this new column
df['train_or_val'] = df['train_or_val'].apply(get_val_rows)

# Filter out training rows
df_train = df[df['train_or_val'] == 'train']
df_train = df_train.drop('train_or_val', axis = 1, inplace = False)
print(len(df_train))
print(len(df_val))

/tmp/ipykernel_34/2742417337.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['train_or_val'] = df['image']


8480
1120


/tmp/ipykernel_34/2742417337.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['train_or_val'] = df['train_or_val'].apply(get_val_rows)


In [21]:
df_train['cell_type_idx'].value_counts()

# Duplicate rows to balance the number of rows in 7 classes..
data_aug_rate = [6.3,1.025,13,19.8,6.1,56.4,47.7]

# Iterate over unique values of 'cell_type_idx'
for i in df_train['cell_type_idx'].unique():

    if data_aug_rate[i] > 1:

        # Filter the DataFrame for the current value of 'cell_type_idx'
        filtered_df = df_train[df_train['cell_type_idx'] == i]

        # Duplicate rows based on the data augmentation rate for this value of 'cell_type_idx'
        duplicated_rows = filtered_df.sample(frac=data_aug_rate[i] - 1, replace=True)

        # Concatenate the original DataFrame with the duplicated rows
        df_train = pd.concat([df_train, duplicated_rows], ignore_index=True)

df_train['cell_type_idx'].value_counts()

cell_type_idx
0    6067
6    5963
2    5954
5    5922
4    5893
3    5881
1    5705
Name: count, dtype: int64

<h1>DataSet Spliiting</h1>

In [23]:
len(df_train)

41385

In [24]:
# Split the test set again in a validation set and a true test set:

df_val, df_test = train_test_split(df_val, test_size=0.5)
df_train = df_train.reset_index()
df_val = df_val.reset_index()
df_test = df_test.reset_index()

In [25]:
print(len(df_test))
df_test['cell_type_idx'].value_counts()

560


cell_type_idx
1    453
4     40
2     23
0     22
3     16
6      5
5      1
Name: count, dtype: int64

<h1>Define Model</h1>

In [86]:
# feature_extract is a boolean that defines finetuning or feature extracting.
# If feature_extract = False, the model is finetuned and all model parameters are updated.
# If feature_extract = True, only the last layer parameters are updated, the others remain fixed.

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [87]:

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18, resnet34, resnet50, resnet101
        """
        model_ft = models.resnet101(pretrained=True)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "mobilenet":
        model_ft = models.mobilenet_v2(pretrained=True, progress=True)
        set_parameter_requires_grad(model_ft, feature_extract)
        input_size = 224
        
    elif model_name == "efficientnet":
        model_ft = models.efficientnet_b3(pretrained=True, progress=True)
        set_parameter_requires_grad(model_ft, feature_extract)
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

In [88]:
torch.cuda.is_available()

True

In [105]:
# Define a new model variable..

model_name = "efficientnet"
num_classes = 7
feature_extract = False
# Initialize the model
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Define the device:
device = torch.device('cuda:0')
# device = torch.device('cpu') # If using cpu
# Put the model on the device:
model = model_ft.to(device)

In [106]:
import torchvision
print(torchvision.__version__)

0.16.2


<H1>Augmenting Data...</H1>

In [107]:
# define the transformation of the train images.
train_transform = transforms.Compose([transforms.Resize((input_size,input_size)),transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomRotation(20),
                                      transforms.ColorJitter(brightness=0.1, contrast=0.1, hue=0.1),
                                      transforms.ToTensor(), 
                                      transforms.Normalize(norm_mean, norm_std)])


# define the transformation of the val images.
val_transform = transforms.Compose([transforms.Resize((input_size,input_size)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(norm_mean, norm_std)])

# define the transformation of the test images.
test_transform = transforms.Compose([transforms.Resize((input_size,input_size)),
                                     transforms.ToTensor(),
                                    transforms.Normalize(norm_mean, norm_std)])

In [108]:
# Define a pytorch dataloader for dataset..
class HAM10000(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Load data and get label
        X = Image.open(self.df['path'][index])
        y = torch.tensor(int(self.df['cell_type_idx'][index]))

        if self.transform:
            X = self.transform(X)

        return X, y

In [109]:
# Define the training set using the table train_df and using the defined transitions (train_transform)
training_set = HAM10000(df_train, transform=train_transform)
train_loader = DataLoader(training_set, batch_size=32, shuffle=True, num_workers=4)

# Same for the validation set:
validation_set = HAM10000(df_val, transform=train_transform)
val_loader = DataLoader(validation_set, batch_size=32, shuffle=False, num_workers=4)

# Same for the test set:
test_set = HAM10000(df_test, transform=train_transform)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False, num_workers=4)

In [110]:
# this function is used during training process, to calculate the loss and accuracy
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [111]:
total_loss_train, total_acc_train = [],[]

def train(train_loader, model, criterion, optimizer, epoch, threshold_batch):
    model.train()
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    curr_iter = (epoch - 1) * len(train_loader)
    for i, data in enumerate(train_loader):
        
        if threshold_batch and i>= threshold_batch:
            break       

        images, labels = data
        N = images.size(0)
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        prediction = outputs.max(1, keepdim=True)[1]
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
        train_loss.update(loss.item())
        curr_iter += 1
        if (i + 1) % 100 == 0:
            print('[epoch %d], [iter %d / %d], [train loss %.5f], [train acc %.5f]' % (
                epoch, i + 1, len(train_loader), train_loss.avg, train_acc.avg))
            total_loss_train.append(train_loss.avg)
            total_acc_train.append(train_acc.avg)
    return train_loss.avg, train_acc.avg

In [112]:
def validate(val_loader, model, criterion, optimizer, epoch):
    model.eval()
    val_loss = AverageMeter()
    val_acc = AverageMeter()

    with torch.no_grad():
        for i, data in tqdm(enumerate(val_loader)):

            images, labels = data
            N = images.size(0)
            images = Variable(images).to(device)
            labels = Variable(labels).to(device)

            outputs = model(images)
            prediction = outputs.max(1, keepdim=True)[1]

            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
            val_loss.update(criterion(outputs, labels).item())

    print('------------------------------------------------------------')
    print('[epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss.avg, val_acc.avg))
    print('------------------------------------------------------------')
    return val_loss.avg, val_acc.avg

In [113]:
# Start Training..

def train_model(epoch_num, lr, optimizer_func = optim.Adam, threshold_batch = None, criterion = nn.CrossEntropyLoss()):

    best_val_acc = 0
    total_loss_val, total_acc_val = [],[]
    for epoch in tqdm(range(1, epoch_num+1)):

        # set optimizer and loss function
        optimizer = optimizer_func(model.parameters(), lr= lr)
        criterion = criterion.to(device)

        loss_train, acc_train = train(train_loader, model, criterion, optimizer, epoch, threshold_batch)
        loss_val, acc_val = validate(val_loader, model, criterion, optimizer, epoch)
        total_loss_val.append(loss_val)
        total_acc_val.append(acc_val)

In [114]:
# Training - 1st
train_model(1,1e-3)

  0%|          | 0/1 [00:00<?, ?it/s]

[epoch 1], [iter 100 / 1294], [train loss 1.41863], [train acc 0.57281]
[epoch 1], [iter 200 / 1294], [train loss 1.10423], [train acc 0.64312]
[epoch 1], [iter 300 / 1294], [train loss 0.95013], [train acc 0.68437]
[epoch 1], [iter 400 / 1294], [train loss 0.85662], [train acc 0.71023]
[epoch 1], [iter 500 / 1294], [train loss 0.78896], [train acc 0.73000]
[epoch 1], [iter 600 / 1294], [train loss 0.73936], [train acc 0.74484]
[epoch 1], [iter 700 / 1294], [train loss 0.69595], [train acc 0.75795]
[epoch 1], [iter 800 / 1294], [train loss 0.66222], [train acc 0.76988]
[epoch 1], [iter 900 / 1294], [train loss 0.63065], [train acc 0.77965]
[epoch 1], [iter 1000 / 1294], [train loss 0.60678], [train acc 0.78781]
[epoch 1], [iter 1100 / 1294], [train loss 0.58229], [train acc 0.79577]
[epoch 1], [iter 1200 / 1294], [train loss 0.56044], [train acc 0.80263]



0it [00:00, ?it/s]
1it [00:00,  1.02it/s]
3it [00:01,  3.27it/s]
5it [00:01,  3.12it/s]
7it [00:01,  4.71it/s]
9it [00:02,  3.87it/s]
11it [00:02,  5.24it/s]
13it [00:03,  4.21it/s]
15it [00:03,  5.53it/s]
18it [00:03,  4.50it/s]
100%|██████████| 1/1 [05:09<00:00, 309.50s/it]

------------------------------------------------------------
[epoch 1], [val loss 0.27333], [val acc 0.90972]
------------------------------------------------------------


In [115]:
# Check results on test dataset..
validate(test_loader, model, criterion, optimizer, 1)

18it [00:03,  4.67it/s]

------------------------------------------------------------
[epoch 1], [val loss 0.38759], [val acc 0.87847]
------------------------------------------------------------


(0.3875850381122695, 0.8784722222222222)

In [116]:
# Training - 2nd..
train_model(1,1e-5,threshold_batch = 300)

  0%|          | 0/1 [00:00<?, ?it/s]

[epoch 1], [iter 100 / 1294], [train loss 0.25317], [train acc 0.91187]
[epoch 1], [iter 200 / 1294], [train loss 0.24121], [train acc 0.91453]
[epoch 1], [iter 300 / 1294], [train loss 0.23636], [train acc 0.91458]



0it [00:00, ?it/s]
1it [00:00,  1.14it/s]
3it [00:01,  3.60it/s]
5it [00:01,  3.35it/s]
7it [00:01,  4.87it/s]
9it [00:02,  4.04it/s]
11it [00:02,  5.43it/s]
13it [00:03,  4.33it/s]
15it [00:03,  5.60it/s]
18it [00:03,  4.59it/s]
100%|██████████| 1/1 [01:16<00:00, 76.79s/it]

------------------------------------------------------------
[epoch 1], [val loss 0.29329], [val acc 0.88889]
------------------------------------------------------------


In [117]:
# Check results on test dataset..
validate(test_loader, model, criterion, optimizer, 1)

18it [00:03,  4.62it/s]

------------------------------------------------------------
[epoch 1], [val loss 0.36245], [val acc 0.88194]
------------------------------------------------------------


(0.3624495201640659, 0.8819444444444444)

In [122]:
# Training - 3rd..
train_model(1,1e-5,threshold_batch = 300)

  0%|          | 0/1 [00:00<?, ?it/s]

[epoch 1], [iter 100 / 1294], [train loss 0.20879], [train acc 0.92375]
[epoch 1], [iter 200 / 1294], [train loss 0.20811], [train acc 0.92078]
[epoch 1], [iter 300 / 1294], [train loss 0.20826], [train acc 0.92323]



0it [00:00, ?it/s]
1it [00:00,  1.18it/s]
3it [00:00,  3.65it/s]
5it [00:01,  3.00it/s]
7it [00:01,  4.41it/s]
9it [00:02,  3.88it/s]
11it [00:02,  5.19it/s]
13it [00:03,  4.26it/s]
15it [00:03,  5.58it/s]
18it [00:03,  4.54it/s]
100%|██████████| 1/1 [01:16<00:00, 76.57s/it]

------------------------------------------------------------
[epoch 1], [val loss 0.27672], [val acc 0.90451]
------------------------------------------------------------


In [123]:
# Check results on test dataset..
validate(test_loader, model, criterion, optimizer, 1)

18it [00:03,  4.73it/s]

------------------------------------------------------------
[epoch 1], [val loss 0.34214], [val acc 0.87847]
------------------------------------------------------------


(0.342144925147295, 0.8784722222222222)

<h1> Predictions... </h1>

In [118]:
# Get data..
img_data_dir = data_dir + r"/KCDH2024_Test_Input/KCDH2024_Test_Input"
all_image_path = glob(os.path.join(img_data_dir, '*.jpg'))
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}  # key - image filename,   value - path to image
len(imageid_path_dict)

1512

In [124]:
# Header of txt file..
text = "ID,Class\n"

# Rows ..
model.eval()
with torch.no_grad():
    for imagename, path in tqdm(imageid_path_dict.items()):
         X = Image.open(path)
         X = test_transform(X)
         X = Variable(X).to(device)
         output = model(X.unsqueeze(0))
         prediction = output.max(1, keepdim=True)[1].item()
         text += f"{imagename},{prediction}\n"

100%|██████████| 1512/1512 [00:39<00:00, 38.38it/s]


In [126]:
with open(f'predictions.csv', 'w') as txtfile:
    txtfile.write(text)